In [ ]:
import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import colorcet as cc
from bokeh.tile_providers import STAMEN_TERRAIN_RETINA
from vis_tools import read_hysplit_netcdf
from bokeh.models.glyphs import Image
from bokeh.models.renderers import GlyphRenderer

hv.extension('bokeh')
renderer = hv.renderer('bokeh')

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '18043015_white_island_5.0_0.01.nc'

# SPECIFY:
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
NUM_BINS = 256  # choose 1, 2, 4, 8, or 256 color bins
###############################################
       
model = read_hysplit_netcdf(FILENAME)
model['total_deposition'].values = np.log10(model['total_deposition'].values)  # manually take the log

gv_ds = gv.Dataset(model, crs=ccrs.PlateCarree())
gv_ds = gv_ds.redim.range(total_deposition=(np.log10(ASH_MIN), np.log10(ASH_MAX)))

volc_loc = (model.attrs['volcano_location'][1], model.attrs['volcano_location'][0])
                           
fig = gv.WMTS(STAMEN_TERRAIN_RETINA) * gv_ds.to(gv.Image, ['lon', 'lat'])\
                                     * gv.Points(volc_loc, crs=ccrs.Geodetic(), label='source')
    
# hook function for Bokeh plot adjustments
def adjust_plot(plot, element):
    p = plot.handles['plot']
    
#     renderers = {}  # build a dictionary of Bokeh plot renderers (useful for troubleshooting)
#     for object in p.renderers:
#         key = str(object).split('(')[0] + '_1'
#         i = 2
#         while key in renderers:
#             key = key.split('_')[0] + '_' + str(i)
#             i = i + 1
#         renderers[key] = object      
#     print(renderers.keys())
    
    # modify tools
    tools = dict(zip(map(lambda tool: str(tool).split('(')[0], p.tools), p.tools))
    wz = tools['WheelZoomTool']
    wz.zoom_on_axis = False
    p.tools = [tools['PanTool'], tools['BoxZoomTool'], wz, tools['ResetTool']]
    p.toolbar.active_scroll = wz
       
    # set the global alpha for all Bokeh Image objects
    for object in p.renderers:    
        if isinstance(object, GlyphRenderer):
            if isinstance(object.glyph, Image):
                object.glyph.global_alpha = 0.5
                
    return None
       
cbar_title = 'log\N{SUBSCRIPT ONE}\N{SUBSCRIPT ZERO} [ ash thickness (mm) ] '
plot_opts = {'Image': {'style': dict(cmap=cc.b_linear_kry_5_98_c75[::-(int(256/NUM_BINS))]),\
                        'plot': dict(colorbar=True, title_format=FILENAME.split('/')[-1], height=600, width=700,\
                                     colorbar_opts=dict(location=(0,0), orientation='horizontal',\
                                                        title=cbar_title),\
                                     colorbar_position='bottom', finalize_hooks=[adjust_plot])},\
            'Points': {'style': dict(size=15, marker='triangle', line_color='black', fill_color='cyan'),\
                        'plot': dict()}
            }             
fig = fig.opts(plot_opts)

fig
#renderer.save(fig, '/opt/shared_nbs/ashfall_visual/bokeh_html/geoviews_bokeh_map'); print('HTML file saved')